In [2]:
## Original Version by Ziyu 

import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T

# -----------------------
# Preprocessing functions
# -----------------------

def preprocess_pixel(path, size=(224, 224)):
    """Preprocess for pixel-level similarity: grayscale, resize, normalize."""
    img = Image.open(path).convert("L")
    img = img.resize(size)
    arr = np.array(img, dtype=np.float32)
    arr = (arr - arr.mean()) / (arr.std() + 1e-6)  # normalize
    return arr.flatten()[None, :]  # shape (1, D)

# Pretrained CNN (ResNet-18 for simplicity)
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # remove final classifier, keep features
resnet.eval()

transform_cnn = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

def preprocess_cnn(path):
    """Preprocess for CNN features: resize, 3 channels, normalize for ResNet."""
    img = Image.open(path).convert("RGB")
    x = transform_cnn(img).unsqueeze(0)  # shape (1,3,224,224)
    with torch.no_grad():
        feat = resnet(x).numpy()
    return feat  # shape (1, D)

# -----------------------
# Similarity + Prediction
# -----------------------

def cosine(a, b):
    return cosine_similarity(a, b)[0, 0]

def predict(htr_path, rt13_path, test_path, alpha=0.5):
    # Pixel features
    x_htr_pix = preprocess_pixel(htr_path)
    x_rt13_pix = preprocess_pixel(rt13_path)
    x_test_pix = preprocess_pixel(test_path)

    s_htr_pix = cosine(x_test_pix, x_htr_pix)
    s_rt13_pix = cosine(x_test_pix, x_rt13_pix)

    # CNN features
    x_htr_cnn = preprocess_cnn(htr_path)
    x_rt13_cnn = preprocess_cnn(rt13_path)
    x_test_cnn = preprocess_cnn(test_path)

    s_htr_cnn = cosine(x_test_cnn, x_htr_cnn)
    s_rt13_cnn = cosine(x_test_cnn, x_rt13_cnn)

    # Combined similarity
    s_htr = alpha * s_htr_cnn + (1 - alpha) * s_htr_pix
    s_rt13 = alpha * s_rt13_cnn + (1 - alpha) * s_rt13_pix

    label = "htr" if s_htr > s_rt13 else "rt13"

    return {
        "prediction": label,
        "s_htr_pixel": s_htr_pix,
        "s_rt13_pixel": s_rt13_pix,
        "s_htr_cnn": s_htr_cnn,
        "s_rt13_cnn": s_rt13_cnn,
        "s_htr_combined": s_htr,
        "s_rt13_combined": s_rt13
    }

# -----------------------
# Example run (Colab)
# -----------------------
result = predict("htr.bmp", "rt13.bmp", "test.bmp", alpha=0.5)
print(result)

/Users/justinmeng/.pyenv/versions/3.9.19/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/justinmeng/.pyenv/versions/3.9.19/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: 'htr.bmp'

In [3]:
import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T

# -----------------------
# Preprocessing functions
# -----------------------

def preprocess_pixel(path, size=(224, 224)):
    """Preprocess for pixel-level similarity: grayscale, resize, normalize."""
    img = Image.open(path).convert("L")
    img = img.resize(size)
    arr = np.array(img, dtype=np.float32)
    arr = (arr - arr.mean()) / (arr.std() + 1e-6)  # normalize
    return arr.flatten()[None, :]  # shape (1, D)

# Pretrained CNN (ResNet-18 for simplicity)
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # remove final classifier, keep features
resnet.eval()

transform_cnn = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

def preprocess_cnn(path):
    """Preprocess for CNN features: resize, 3 channels, normalize for ResNet."""
    img = Image.open(path).convert("RGB")
    x = transform_cnn(img).unsqueeze(0)  # shape (1,3,224,224)
    with torch.no_grad():
        feat = resnet(x).numpy()
    return feat  # shape (1, D)

# -----------------------
# Similarity + Prediction
# -----------------------

def cosine(a, b):
    return cosine_similarity(a, b)[0, 0]

def predict(htr_path, rt13_path, test_path, alpha=0.5):
    # Pixel features
    x_htr_pix = preprocess_pixel(htr_path)
    x_rt13_pix = preprocess_pixel(rt13_path)
    x_test_pix = preprocess_pixel(test_path)

    s_htr_pix = cosine(x_test_pix, x_htr_pix)
    s_rt13_pix = cosine(x_test_pix, x_rt13_pix)

    # CNN features
    x_htr_cnn = preprocess_cnn(htr_path)
    x_rt13_cnn = preprocess_cnn(rt13_path)
    x_test_cnn = preprocess_cnn(test_path)

    s_htr_cnn = cosine(x_test_cnn, x_htr_cnn)
    s_rt13_cnn = cosine(x_test_cnn, x_rt13_cnn)

    # Combined similarity
    s_htr = alpha * s_htr_cnn + (1 - alpha) * s_htr_pix
    s_rt13 = alpha * s_rt13_cnn + (1 - alpha) * s_rt13_pix

    label = "htr" if s_htr > s_rt13 else "rt13"

    return {
        "prediction": label,
        "s_htr_pixel": s_htr_pix,
        "s_rt13_pixel": s_rt13_pix,
        "s_htr_cnn": s_htr_cnn,
        "s_rt13_cnn": s_rt13_cnn,
        "s_htr_combined": s_htr,
        "s_rt13_combined": s_rt13
    }

# -----------------------
# Example run (Colab)
# -----------------------
result = predict("htr.bmp", "rt13.bmp", "test.bmp", alpha=0.5)
print(result)

/Users/justinmeng/.pyenv/versions/3.9.19/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/justinmeng/.pyenv/versions/3.9.19/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: 'htr.bmp'

In [12]:
# STO RHEED classifier (HTR vs √13) — Notebook-friendly version
# - ResNet-18 grayscale→3ch embeddings + FFT radial profile
# - Center-disk masking
# - Prototype bank + thresholds (τ margin, κ min-sim)
# - Tune on Val if provided; else leave-one-out (LOO)
# - Functions can be called directly in a notebook
# - Optional CLI if executed as a script

import os, glob, json, math, csv, argparse
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import nnls

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T

# ---------------------------
# Device & model (global)
# ---------------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def _init_resnet18():
    resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    resnet.fc = nn.Identity()
    resnet.to(DEVICE).eval()
    return resnet

RESNET = _init_resnet18()

# Grayscale → 3ch + simple gray stats (domain-robust)
TRANS_GRAY3 = T.Compose([
    T.Resize((224, 224)),
    T.Grayscale(num_output_channels=3),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
])

# ---------------------------
# Utilities & preprocessing
# ---------------------------
def l2norm(x: np.ndarray, axis=1, eps=1e-12):
    return x / (np.linalg.norm(x, axis=axis, keepdims=True) + eps)

def list_images(folder: Optional[str]) -> List[str]:
    if not folder:
        return []
    exts = ("*.png", "*.bmp", "*.jpg", "*.jpeg", "*.tif", "*.tiff")
    paths: List[str] = []
    for e in exts:
        paths += glob.glob(os.path.join(folder, e))
    return sorted(paths)

def imread_gray(path: str) -> np.ndarray:
    return np.array(Image.open(path).convert('L'), dtype=np.float32)

def apply_center_mask(gray: np.ndarray, frac: float = 0.60) -> np.ndarray:
    """
    Mask out the central bright disk; keep annulus.
    frac is the fraction of the shorter side used as mask diameter.
    """
    h, w = gray.shape
    cy, cx = h / 2.0, w / 2.0
    R = min(h, w) * frac * 0.5  # radius
    yy, xx = np.ogrid[:h, :w]
    mask = ((yy - cy) ** 2 + (xx - cx) ** 2) > R ** 2
    out = gray.copy()
    out[~mask] = np.median(gray)
    return out

def fft_radial_profile(gray_224: np.ndarray, n_bins: int = 128) -> np.ndarray:
    """
    Normalized radial power spectrum (1, n_bins).
    gray_224 should already be 224x224 for consistency.
    """
    g = (gray_224 - gray_224.mean()) / (gray_224.std() + 1e-6)
    F = np.fft.fftshift(np.fft.fft2(g))
    P = np.abs(F) ** 2
    h, w = P.shape
    cy, cx = h / 2.0, w / 2.0
    yy, xx = np.indices(P.shape)
    r = np.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r_norm = r / r.max()
    bins = np.linspace(0, 1, n_bins + 1)
    idx = np.digitize(r_norm.ravel(), bins) - 1
    prof = np.array([P.ravel()[idx == i].mean() if np.any(idx == i) else 0.0 for i in range(n_bins)])
    prof = prof / (prof.max() + 1e-12)
    return prof[None, :]

def resnet_features_from_path(path: str) -> np.ndarray:
    img = Image.open(path)
    x = TRANS_GRAY3(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        feat = RESNET(x)  # (1, D)
    f = feat.cpu().numpy()
    return l2norm(f, axis=1)

def extract_features(path: str, mask_center: bool = True, fft_bins: int = 128) -> np.ndarray:
    """
    Unified feature: [ResNet_embed || FFT_radial_profile], L2-normalized.
    """
    # CNN embed (grayscale→3ch)
    f_cnn = resnet_features_from_path(path)  # (1, D1)

    # FFT radial on masked grayscale resized to 224
    gray = imread_gray(path)
    if mask_center:
        gray = apply_center_mask(gray)
    gray224 = np.array(Image.fromarray(gray).resize((224, 224)))
    f_fft = fft_radial_profile(gray224, n_bins=fft_bins)  # (1, 128)

    f = np.concatenate([f_cnn, f_fft], axis=1)
    return l2norm(f, axis=1)

# ---------------------------
# Bank data classes
# ---------------------------
@dataclass
class BankMeta:
    fft_bins: int
    mask_center: bool
    feature_dim: int
    tau_margin: float
    kappa_minsim: float
    temperature: float
    rt13_files: List[str]
    htr_files: List[str]

@dataclass
class PrototypeBank:
    mu_rt13: np.ndarray     # (D,)
    mu_htr:  np.ndarray     # (D,)
    E_rt13:  np.ndarray     # (N_rt13, D)
    E_htr:   np.ndarray     # (N_htr, D)
    meta:    BankMeta

    def save(self, out_npz: str):
        os.makedirs(os.path.dirname(out_npz), exist_ok=True) if os.path.dirname(out_npz) else None
        np.savez_compressed(
            out_npz,
            mu_rt13=self.mu_rt13,
            mu_htr=self.mu_htr,
            E_rt13=self.E_rt13,
            E_htr=self.E_htr,
            meta=json.dumps(asdict(self.meta))
        )

    @staticmethod
    def load(path_npz: str) -> "PrototypeBank":
        z = np.load(path_npz, allow_pickle=True)
        meta = BankMeta(**json.loads(str(z["meta"])))
        return PrototypeBank(
            mu_rt13=z["mu_rt13"],
            mu_htr=z["mu_htr"],
            E_rt13=z["E_rt13"],
            E_htr=z["E_htr"],
            meta=meta
        )

# ---------------------------
# Threshold tuning (Val or LOO)
# ---------------------------
def _grid_f1_tau_kappa(sim_pairs: List[Tuple[float, float, int]]) -> Tuple[float, float]:
    """
    sim_pairs: list of (s_rt13, s_htr, y_true) with y_true in {1 (rt13), 0 (htr)}.
    Returns best (tau, kappa) by macro-F1, counting abstain as wrong.
    """
    taus   = np.linspace(0.02, 0.15, 8)
    kappas = np.linspace(0.15, 0.35, 9)
    best_f1, best = -1.0, (0.08, 0.25)
    y_true = [y for _, _, y in sim_pairs]
    for t in taus:
        for k in kappas:
            y_pred = []
            for sr, sh, _ in sim_pairs:
                smax = max(sr, sh); margin = abs(sr - sh)
                if smax < k or margin < t:
                    y_pred.append(-1)  # abstain → treat as wrong for tuning
                else:
                    y_pred.append(1 if sr > sh else 0)
            tp = sum((yt==1 and yp==1) for yt, yp in zip(y_true, y_pred))
            tn = sum((yt==0 and yp==0) for yt, yp in zip(y_true, y_pred))
            fp = sum((yt==0 and yp==1) for yt, yp in zip(y_true, y_pred))
            fn = sum((yt==1 and yp==0) for yt, yp in zip(y_true, y_pred))
            prec_pos = tp / max(1, tp+fp);  rec_pos = tp / max(1, tp+fn)
            prec_neg = tn / max(1, tn+fn);  rec_neg = tn / max(1, tn+fp)
            f1_pos = 2*prec_pos*rec_pos / max(1e-12, (prec_pos+rec_pos))
            f1_neg = 2*prec_neg*rec_neg / max(1e-12, (prec_neg+rec_neg))
            f1_macro = 0.5 * (f1_pos + f1_neg)
            if f1_macro > best_f1:
                best_f1, best = f1_macro, (t, k)
    return best

def _tune_thresholds_on_val(mu_rt13: np.ndarray, mu_htr: np.ndarray,
                            val_rt13_paths: List[str], val_htr_paths: List[str],
                            mask_center=True, fft_bins=128) -> Optional[Tuple[float,float]]:
    if not val_rt13_paths or not val_htr_paths:
        return None
    sims = []
    for p in val_rt13_paths:
        f = extract_features(p, mask_center=mask_center, fft_bins=fft_bins)
        sr = float(cosine_similarity(f, mu_rt13[None, :])[0, 0])
        sh = float(cosine_similarity(f, mu_htr [None, :])[0, 0])
        sims.append((sr, sh, 1))
    for p in val_htr_paths:
        f = extract_features(p, mask_center=mask_center, fft_bins=fft_bins)
        sr = float(cosine_similarity(f, mu_rt13[None, :])[0, 0])
        sh = float(cosine_similarity(f, mu_htr [None, :])[0, 0])
        sims.append((sr, sh, 0))
    return _grid_f1_tau_kappa(sims)

# ---------------------------
# Build bank (notebook function)
# ---------------------------
def build_bank_from_dirs(rt13_dir: str,
                         htr_dir: str,
                         out_npz: Optional[str] = None,
                         fft_bins: int = 128,
                         mask_center: bool = True,
                         val_rt13_dir: Optional[str] = None,
                         val_htr_dir: Optional[str] = None) -> PrototypeBank:
    rt13_paths = list_images(rt13_dir)
    htr_paths  = list_images(htr_dir)
    if not rt13_paths or not htr_paths:
        raise FileNotFoundError("No exemplar images found in provided exemplar folders.")

    E_rt13 = np.vstack([extract_features(p, mask_center=mask_center, fft_bins=fft_bins)[0] for p in rt13_paths])
    E_htr  = np.vstack([extract_features(p, mask_center=mask_center, fft_bins=fft_bins)[0] for p in htr_paths])

    mu_rt13 = l2norm(E_rt13.mean(axis=0, keepdims=True), axis=1)[0]
    mu_htr  = l2norm(E_htr.mean(axis=0,  keepdims=True), axis=1)[0]

    # thresholds: try Val first; fallback to LOO
    tau, kappa, temp = 0.08, 0.25, 0.50
    tuned = None
    val_rt13_paths = list_images(val_rt13_dir)
    val_htr_paths  = list_images(val_htr_dir)
    if val_rt13_paths and val_htr_paths:
        tuned = _tune_thresholds_on_val(mu_rt13, mu_htr, val_rt13_paths, val_htr_paths,
                                        mask_center=mask_center, fft_bins=fft_bins)
    if tuned is None:
        # LOO on exemplars
        sims = []
        all_pairs = [(p, 1) for p in rt13_paths] + [(p, 0) for p in htr_paths]
        for p, y in all_pairs:
            f = extract_features(p, mask_center=mask_center, fft_bins=fft_bins)
            sr = float(cosine_similarity(f, mu_rt13[None, :])[0, 0])
            sh = float(cosine_similarity(f, mu_htr [None, :])[0, 0])
            sims.append((sr, sh, y))
        tau, kappa = _grid_f1_tau_kappa(sims)
    else:
        tau, kappa = tuned

    meta = BankMeta(
        fft_bins=fft_bins,
        mask_center=mask_center,
        feature_dim=int(len(mu_rt13)),
        tau_margin=float(tau),
        kappa_minsim=float(kappa),
        temperature=0.50,
        rt13_files=rt13_paths,
        htr_files=htr_paths,
    )
    bank = PrototypeBank(mu_rt13=mu_rt13, mu_htr=mu_htr, E_rt13=E_rt13, E_htr=E_htr, meta=meta)

    if out_npz:
        bank.save(out_npz)
        print(f"[OK] bank saved -> {out_npz}")
        print(f"     tau_margin={bank.meta.tau_margin:.3f}, kappa_minsim={bank.meta.kappa_minsim:.3f}, temperature={bank.meta.temperature:.2f}")
        print(f"     rt13 exemplars: {len(bank.meta.rt13_files)}, htr exemplars: {len(bank.meta.htr_files)}")
    else:
        print(f"[OK] bank built (not saved). tau={tau:.3f}, kappa={kappa:.3f}")
    return bank

# ---------------------------
# Prediction helpers
# ---------------------------
def softmax_scores(scores: np.ndarray, T: float = 1.0) -> np.ndarray:
    z = (scores / max(1e-12, T))
    z = z - z.max()
    e = np.exp(z)
    return e / (e.sum() + 1e-12)

def mixture_weights_nnls(f: np.ndarray, protos: List[np.ndarray]) -> np.ndarray:
    P = np.stack(protos, axis=1)  # (D, C)
    w, _ = nnls(P, f)             # (C,)
    s = w.sum()
    return (w / s) if s > 1e-12 else np.ones_like(w)/len(w)

def predict_one(image_path: str, bank: PrototypeBank) -> Dict:
    f = extract_features(image_path, mask_center=bank.meta.mask_center, fft_bins=bank.meta.fft_bins)
    s_rt13 = float(cosine_similarity(f, bank.mu_rt13[None, :])[0, 0])
    s_htr  = float(cosine_similarity(f, bank.mu_htr [None, :])[0, 0])
    s_max  = max(s_rt13, s_htr)
    margin = abs(s_rt13 - s_htr)

    probs = softmax_scores(np.array([s_rt13, s_htr]), T=bank.meta.temperature)
    p_rt13, p_htr = float(probs[0]), float(probs[1])

    if s_max < bank.meta.kappa_minsim or margin < bank.meta.tau_margin:
        label = "uncertain"
    else:
        label = "rt13" if s_rt13 > s_htr else "htr"

    w = mixture_weights_nnls(f[0], [bank.mu_rt13, bank.mu_htr])

    return {
        "file": os.path.basename(image_path),
        "label": label,
        "s_rt13": s_rt13,
        "s_htr":  s_htr,
        "s_max":  s_max,
        "margin": margin,
        "p_rt13": p_rt13,
        "p_htr":  p_htr,
        "w_rt13": float(w[0]),
        "w_htr":  float(w[1]),
    }

def predict_folder_with_bank(input_dir: str, bank: PrototypeBank, out_csv: Optional[str] = None) -> List[Dict]:
    paths = list_images(input_dir)
    if not paths:
        raise FileNotFoundError(f"No images found in '{input_dir}'")
    rows = [predict_one(p, bank) for p in paths]
    if out_csv:
        os.makedirs(os.path.dirname(out_csv), exist_ok=True) if os.path.dirname(out_csv) else None
        with open(out_csv, "w", newline="") as f:
            w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
            w.writeheader()
            for r in rows:
                w.writerow(r)
        print(f"[OK] wrote {len(rows)} predictions -> {out_csv}")
    return rows

# ---------------------------
# Optional CLI (still works if run as a script)
# ---------------------------
def _cli():
    parser = argparse.ArgumentParser(description="STO RHEED classifier (HTR vs √13)")
    sub = parser.add_subparsers(dest="cmd", required=True)

    p_build = sub.add_parser("build-bank", help="Build prototype bank from exemplar folders")
    p_build.add_argument("--rt13", required=True, help="Folder of √13 exemplar images")
    p_build.add_argument("--htr",  required=True, help="Folder of HTR exemplar images")
    p_build.add_argument("--out",  required=True, help="Output .npz path for the bank")
    p_build.add_argument("--fft-bins", type=int, default=128)
    p_build.add_argument("--no-mask", action="store_true", help="Disable center disk masking")
    # NEW: tune on Val if provided
    p_build.add_argument("--val-rt13", default=None, help="(optional) Val folder for √13")
    p_build.add_argument("--val-htr",  default=None, help="(optional) Val folder for HTR")

    p_pred = sub.add_parser("predict", help="Classify images in a folder with a saved bank")
    p_pred.add_argument("--bank", required=True, help="Path to bank .npz")
    p_pred.add_argument("--in-dir", required=True, help="Folder of images to classify")
    p_pred.add_argument("--out-csv", required=True, help="Where to write predictions CSV")

    args = parser.parse_args()

    if args.cmd == "build-bank":
        bank = build_bank_from_dirs(
            rt13_dir=args.rt13,
            htr_dir=args.htr,
            out_npz=args.out,
            fft_bins=args.fft_bins,
            mask_center=not args.no_mask,
            val_rt13_dir=args.val_rt13,
            val_htr_dir=args.val_htr
        )
        # build_bank_from_dirs already prints thresholds & counts

    elif args.cmd == "predict":
        bank = PrototypeBank.load(args.bank)
        predict_folder_with_bank(args.in_dir, bank, args.out_csv)

if __name__ == "__main__":
    # Running as a script (ignored in notebooks unless you call it)
    pass  # replace with: _cli()




bank = build_bank_from_dirs(
    htr_dir="/Users/justinmeng/Desktop/Project Quantum/data/STO_ideal_HTR",
    rt13_dir="/Users/justinmeng/Desktop/Project Quantum/data/STO_ideal_RT13",
    out_npz="artifacts/banks/sto_bank_2025_10_20.npz",  # or None to skip saving
    fft_bins=128,
    mask_center=True,
    val_rt13_dir="/Users/justinmeng/Desktop/Project Quantum/data/Val",   # your Val/ has both classes; that's fine
    val_htr_dir="/Users/justinmeng/Desktop/Project Quantum/data/Val"
)



rows = predict_folder_with_bank(
    input_dir="/Users/justinmeng/Desktop/Project Quantum/data/Test",
    bank=bank,  # or PrototypeBank.load("artifacts/banks/sto_bank_2025_10_20.npz")
    out_csv="artifacts/results/test_preds.csv"
)
rows

[OK] bank saved -> artifacts/banks/sto_bank_2025_10_20.npz
     tau_margin=0.020, kappa_minsim=0.150, temperature=0.50
     rt13 exemplars: 8, htr exemplars: 10
[OK] wrote 4 predictions -> artifacts/results/test_preds.csv


[{'file': 'HTR_12.png',
  'label': 'uncertain',
  's_rt13': 0.963294678674919,
  's_htr': 0.9825009697241804,
  's_max': 0.9825009697241804,
  'margin': 0.019206291049261415,
  'p_rt13': 0.4903980351088602,
  'p_htr': 0.5096019648906301,
  'w_rt13': 0.0,
  'w_htr': 1.0},
 {'file': 'HTR_13.png',
  'label': 'htr',
  's_rt13': 0.9593800400781332,
  's_htr': 0.9826346059437672,
  's_max': 0.9826346059437672,
  'margin': 0.023254565865634014,
  'p_rt13': 0.488374812527663,
  'p_htr': 0.5116251874718253,
  'w_rt13': 0.0,
  'w_htr': 1.0},
 {'file': 'RT13_11.png',
  'label': 'uncertain',
  's_rt13': 0.9861658445474778,
  's_htr': 0.981800948384859,
  's_max': 0.9861658445474778,
  'margin': 0.004364896162618814,
  'p_rt13': 0.5021824342209308,
  'p_htr': 0.49781756577856695,
  'w_rt13': 0.7087100914341642,
  'w_htr': 0.29128990856583586},
 {'file': 'RT13_8.png',
  'label': 'uncertain',
  's_rt13': 0.9751316476790102,
  's_htr': 0.9615956929753822,
  's_max': 0.9751316476790102,
  'margin': 0.0

In [11]:
# =========================
# STO RHEED classifier v2  (Blocks A, B, C implemented)
# =========================

import os, glob, json, math, csv, argparse
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import nnls
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T

# ---------------------------
# Device & model (global)
# ---------------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def _init_resnet18():
    resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    resnet.fc = nn.Identity()
    resnet.to(DEVICE).eval()
    return resnet

RESNET = _init_resnet18()

# Grayscale → 3ch + simple gray stats (domain-robust)
TRANS_GRAY3 = T.Compose([
    T.Resize((224, 224)),
    T.Grayscale(num_output_channels=3),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
])

# ---------------------------
# Utilities & preprocessing
# ---------------------------
def l2norm(x: np.ndarray, axis=1, eps=1e-12):
    return x / (np.linalg.norm(x, axis=axis, keepdims=True) + eps)

def list_images(folder: Optional[str]) -> List[str]:
    if not folder:
        return []
    exts = ("*.png", "*.bmp", "*.jpg", "*.jpeg", "*.tif", "*.tiff")
    paths: List[str] = []
    for e in exts:
        paths += glob.glob(os.path.join(folder, e))
    return sorted(paths)

def imread_gray(path: str) -> np.ndarray:
    return np.array(Image.open(path).convert('L'), dtype=np.float32)

def apply_center_mask(gray: np.ndarray, frac: float = 0.60) -> np.ndarray:
    """
    Mask out the central bright disk; keep annulus.
    frac: fraction of the shorter side used as mask diameter.
    """
    h, w = gray.shape
    cy, cx = h / 2.0, w / 2.0
    R = min(h, w) * frac * 0.5  # radius
    yy, xx = np.ogrid[:h, :w]
    mask = ((yy - cy) ** 2 + (xx - cx) ** 2) > R ** 2
    out = gray.copy()
    out[~mask] = np.median(gray)
    return out

def _percentile_contrast(arr: np.ndarray) -> float:
    p1, p99 = np.percentile(arr, [1, 99])
    return float(p99 - p1)

def _var_laplacian(arr: np.ndarray) -> float:
    from scipy.ndimage import laplace
    x = (arr - arr.min()) / max(1e-6, (arr.max() - arr.min()))
    return float(laplace(x).var())

def _sat_total(arr: np.ndarray, low=5, high=250) -> float:
    n = arr.size
    return float(((arr <= low).sum() + (arr >= high).sum()) / n)

def score_quality(gray: np.ndarray) -> float:
    """Composite quality: high contrast & sharpness, low saturation."""
    c = _percentile_contrast(gray)
    v = _var_laplacian(gray)
    s = _sat_total(gray)
    # normalize roughly by robust scales to balance terms
    c_n = c / 255.0
    v_n = v / (v + 1.0)
    s_n = s  # already 0..1-ish fraction
    return 0.4 * c_n + 0.5 * v_n + 0.1 * (1.0 - s_n)

def fft_radial_profile(gray_224: np.ndarray, n_bins: int = 128) -> np.ndarray:
    """
    Normalized radial power spectrum (1, n_bins).
    gray_224 should already be 224x224 for consistency.
    """
    g = (gray_224 - gray_224.mean()) / (gray_224.std() + 1e-6)
    F = np.fft.fftshift(np.fft.fft2(g))
    P = np.abs(F) ** 2
    h, w = P.shape
    cy, cx = h / 2.0, w / 2.0
    yy, xx = np.indices(P.shape)
    r = np.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r_norm = r / r.max()
    bins = np.linspace(0, 1, n_bins + 1)
    idx = np.digitize(r_norm.ravel(), bins) - 1
    prof = np.array([P.ravel()[idx == i].mean() if np.any(idx == i) else 0.0 for i in range(n_bins)])
    prof = prof / (prof.max() + 1e-12)
    return prof[None, :]

def resnet_features_from_path(path: str) -> np.ndarray:
    img = Image.open(path)
    x = TRANS_GRAY3(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        feat = RESNET(x)  # (1, D)
    f = feat.cpu().numpy()
    return l2norm(f, axis=1)


# def extract_features(path: str,
#                      mask_center: bool = True,
#                      fft_bins: int = 128,
#                      gamma_fft: float = 1.5) -> np.ndarray:
#     """
#     Unified feature (Block C): [ResNet_embed || γ * FFT_radial_profile], L2-normalized.
#     """
#     # CNN embed (grayscale→3ch)
#     f_cnn = resnet_features_from_path(path)  # (1, D1)

#     # FFT radial on masked grayscale resized to 224
#     gray = imread_gray(path)
#     if mask_center:
#         gray = apply_center_mask(gray)
#     gray224 = np.array(Image.fromarray(gray).resize((224, 224)))
#     f_fft = fft_radial_profile(gray224, n_bins=fft_bins)  # (1, 128)

#     f = np.concatenate([f_cnn, gamma_fft * f_fft], axis=1)
#     return l2norm(f, axis=1)



def extract_features(path: str, mask_center: bool=True, fft_bins: int=128, gamma_fft: float=1.5):
    # 1) load & preprocess once
    gray = imread_gray(path)
    if mask_center:
        gray = apply_center_mask(gray)          # SAME mask for both branches
    gray224 = np.array(Image.fromarray(gray).resize((224, 224)))

    # 2) CNN on the preprocessed luminance (repeat-to-3ch inside TRANS_GRAY3)
    img_for_cnn = Image.fromarray(gray224.astype(np.uint8))
    x = TRANS_GRAY3(img_for_cnn).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        f_cnn = RESNET(x).cpu().numpy()
    f_cnn = l2norm(f_cnn, axis=1)

    # 3) FFT radial on the same gray224
    f_fft = fft_radial_profile(gray224, n_bins=fft_bins)

    # 4) concat + L2
    f = np.concatenate([f_cnn, gamma_fft * f_fft], axis=1)
    return l2norm(f, axis=1)





# ---------------------------
# Bank data classes
# ---------------------------
@dataclass
class BankMeta:
    fft_bins: int
    mask_center: bool
    feature_dim: int
    tau_margin: float
    kappa_minsim: float
    temperature: float
    gamma_fft: float
    # multi-prototype storage
    rt13_protos: List[np.ndarray]   # list of (D,)
    htr_protos:  List[np.ndarray]   # list of (D,)
    rt13_files: List[str]
    htr_files:  List[str]

@dataclass
class PrototypeBank:
    # For convenience we also keep mean prototypes (first entries), but we score via proto lists.
    mu_rt13: np.ndarray     # (D,)
    mu_htr:  np.ndarray     # (D,)
    E_rt13:  np.ndarray     # (N_rt13, D)
    E_htr:   np.ndarray     # (N_htr, D)
    meta:    BankMeta

    def save(self, out_npz: str):
        os.makedirs(os.path.dirname(out_npz), exist_ok=True) if os.path.dirname(out_npz) else None
        # Convert lists of arrays to a single array with object dtype for saving
        rt13P = np.stack(self.meta.rt13_protos, axis=0) if len(self.meta.rt13_protos) else np.zeros((0, self.mu_rt13.shape[0]))
        htrP  = np.stack(self.meta.htr_protos,  axis=0) if len(self.meta.htr_protos)  else np.zeros((0, self.mu_htr.shape[0]))
        meta_dict = asdict(self.meta).copy()
        # remove numpy arrays (we save separately)
        meta_dict.pop('rt13_protos'); meta_dict.pop('htr_protos')
        np.savez_compressed(
            out_npz,
            mu_rt13=self.mu_rt13,
            mu_htr=self.mu_htr,
            E_rt13=self.E_rt13,
            E_htr=self.E_htr,
            rt13_protos=rt13P,
            htr_protos=htrP,
            meta=json.dumps(meta_dict)
        )

    @staticmethod
    def load(path_npz: str) -> "PrototypeBank":
        z = np.load(path_npz, allow_pickle=True)
        meta = json.loads(str(z["meta"]))
        rt13P = z["rt13_protos"]; htrP = z["htr_protos"]
        meta_obj = BankMeta(
            fft_bins=int(meta["fft_bins"]),
            mask_center=bool(meta["mask_center"]),
            feature_dim=int(meta["feature_dim"]),
            tau_margin=float(meta["tau_margin"]),
            kappa_minsim=float(meta["kappa_minsim"]),
            temperature=float(meta["temperature"]),
            gamma_fft=float(meta["gamma_fft"]),
            rt13_protos=[rt13P[i] for i in range(len(rt13P))],
            htr_protos=[htrP[i] for i in range(len(htrP))],
            rt13_files=list(meta["rt13_files"]),
            htr_files=list(meta["htr_files"]),
        )
        return PrototypeBank(
            mu_rt13=z["mu_rt13"],
            mu_htr=z["mu_htr"],
            E_rt13=z["E_rt13"],
            E_htr=z["E_htr"],
            meta=meta_obj
        )

# ---------------------------
# Threshold tuning (Val or LOO)
# ---------------------------
def _grid_f1_tau_kappa(sim_pairs: List[Tuple[float, float, int]]) -> Tuple[float, float]:
    """
    sim_pairs: list of (s_rt13, s_htr, y_true) with y_true in {1 (rt13), 0 (htr)}.
    Returns best (tau, kappa) by macro-F1, counting abstain as wrong.
    """
    taus   = np.linspace(0.02, 0.15, 8)
    kappas = np.linspace(0.15, 0.35, 9)
    best_f1, best = -1.0, (0.08, 0.25)
    y_true = [y for _, _, y in sim_pairs]
    for t in taus:
        for k in kappas:
            y_pred = []
            for sr, sh, _ in sim_pairs:
                smax = max(sr, sh); margin = abs(sr - sh)
                if smax < k or margin < t:
                    y_pred.append(-1)  # abstain → treat as wrong for tuning
                else:
                    y_pred.append(1 if sr > sh else 0)
            tp = sum((yt==1 and yp==1) for yt, yp in zip(y_true, y_pred))
            tn = sum((yt==0 and yp==0) for yt, yp in zip(y_true, y_pred))
            fp = sum((yt==0 and yp==1) for yt, yp in zip(y_true, y_pred))
            fn = sum((yt==1 and yp==0) for yt, yp in zip(y_true, y_pred))
            prec_pos = tp / max(1, tp+fp);  rec_pos = tp / max(1, tp+fn)
            prec_neg = tn / max(1, tn+fn);  rec_neg = tn / max(1, tn+fp)
            f1_pos = 2*prec_pos*rec_pos / max(1e-12, (prec_pos+rec_pos))
            f1_neg = 2*prec_neg*rec_neg / max(1e-12, (prec_neg+rec_neg))
            f1_macro = 0.5 * (f1_pos + f1_neg)
            if f1_macro > best_f1:
                best_f1, best = f1_macro, (t, k)
    return best

def _tune_thresholds_on_val(mu_rt13: np.ndarray, mu_htr: np.ndarray,
                            val_rt13_paths: List[str], val_htr_paths: List[str],
                            mask_center=True, fft_bins=128, gamma_fft=1.5) -> Optional[Tuple[float,float]]:
    if not val_rt13_paths or not val_htr_paths:
        return None
    sims = []
    for p in val_rt13_paths:
        f = extract_features(p, mask_center=mask_center, fft_bins=fft_bins, gamma_fft=gamma_fft)
        sr = float(cosine_similarity(f, mu_rt13[None, :])[0, 0])
        sh = float(cosine_similarity(f, mu_htr [None, :])[0, 0])
        sims.append((sr, sh, 1))
    for p in val_htr_paths:
        f = extract_features(p, mask_center=mask_center, fft_bins=fft_bins, gamma_fft=gamma_fft)
        sr = float(cosine_similarity(f, mu_rt13[None, :])[0, 0])
        sh = float(cosine_similarity(f, mu_htr [None, :])[0, 0])
        sims.append((sr, sh, 0))
    return _grid_f1_tau_kappa(sims)

# ---------------------------
# Build bank (Blocks B + C)
# ---------------------------
def _keep_topK_by_quality(paths: List[str], K: Optional[int]) -> List[str]:
    if not K or K >= len(paths):
        return paths
    scored = []
    for p in paths:
        g = imread_gray(p)
        scored.append((score_quality(g), p))
    scored.sort(reverse=True, key=lambda t: t[0])
    return [p for _, p in scored[:K]]

def _compute_prototypes(E: np.ndarray, k_per_class: int) -> List[np.ndarray]:
    """Return list of k prototypes (means or k-means centroids), L2-normalized."""
    if k_per_class <= 1 or E.shape[0] < 3:
        mu = l2norm(E.mean(axis=0, keepdims=True), axis=1)[0]
        return [mu]
    k = min(k_per_class, max(1, E.shape[0] // 2))  # avoid over-clustering
    km = KMeans(n_clusters=k, n_init=10, random_state=0)
    km.fit(E)
    protos = []
    for ci in range(k):
        members = E[km.labels_ == ci]
        if len(members) == 0:
            continue
        mu = l2norm(members.mean(axis=0, keepdims=True), axis=1)[0]
        protos.append(mu)
    if not protos:
        protos = [l2norm(E.mean(axis=0, keepdims=True), axis=1)[0]]
    return protos

def build_bank_from_dirs(rt13_dir: str,
                         htr_dir: str,
                         out_npz: Optional[str] = None,
                         fft_bins: int = 128,
                         mask_center: bool = True,
                         gamma_fft: float = 1.5,         # Block C: FFT gain
                         k_per_class: int = 2,           # Block B: multi-prototype
                         top_k_rt13: Optional[int] = None,  # Block B: keep top-K by quality
                         top_k_htr: Optional[int] = None,
                         val_rt13_dir: Optional[str] = None,
                         val_htr_dir: Optional[str] = None,
                         temperature: float = 0.30       # Block A: sharper display probs
                         ) -> PrototypeBank:
    rt13_paths_all = list_images(rt13_dir)
    htr_paths_all  = list_images(htr_dir)
    if not rt13_paths_all or not htr_paths_all:
        raise FileNotFoundError("No exemplar images found in provided exemplar folders.")

    # Block B: quality curation
    rt13_paths = _keep_topK_by_quality(rt13_paths_all, top_k_rt13)
    htr_paths  = _keep_topK_by_quality(htr_paths_all,  top_k_htr)

    # Extract features
    E_rt13 = np.vstack([extract_features(p, mask_center=mask_center, fft_bins=fft_bins, gamma_fft=gamma_fft)[0]
                        for p in rt13_paths])
    E_htr  = np.vstack([extract_features(p, mask_center=mask_center, fft_bins=fft_bins, gamma_fft=gamma_fft)[0]
                        for p in htr_paths])

    # Mean prototypes (also stored)
    mu_rt13 = l2norm(E_rt13.mean(axis=0, keepdims=True), axis=1)[0]
    mu_htr  = l2norm(E_htr.mean(axis=0,  keepdims=True), axis=1)[0]

    # Block B: multi-prototype (k-means centroids, L2-normalized)
    rt13_protos = _compute_prototypes(E_rt13, k_per_class=k_per_class)
    htr_protos  = _compute_prototypes(E_htr,  k_per_class=k_per_class)

    # thresholds: try Val first; fallback to LOO
    tau, kappa = 0.08, 0.25
    tuned = None
    val_rt13_paths = list_images(val_rt13_dir)
    val_htr_paths  = list_images(val_htr_dir)
    if val_rt13_paths and val_htr_paths:
        tuned = _tune_thresholds_on_val(mu_rt13, mu_htr, val_rt13_paths, val_htr_paths,
                                        mask_center=mask_center, fft_bins=fft_bins, gamma_fft=gamma_fft)
    if tuned is None:
        # LOO on exemplars (still using mean prototypes for tuning simplicity)
        sims = []
        all_pairs = [(p, 1) for p in rt13_paths] + [(p, 0) for p in htr_paths]
        for p, y in all_pairs:
            f = extract_features(p, mask_center=mask_center, fft_bins=fft_bins, gamma_fft=gamma_fft)
            sr = float(cosine_similarity(f, mu_rt13[None, :])[0, 0])
            sh = float(cosine_similarity(f, mu_htr [None, :])[0, 0])
            sims.append((sr, sh, y))
        tau, kappa = _grid_f1_tau_kappa(sims)
    else:
        tau, kappa = tuned

    meta = BankMeta(
        fft_bins=fft_bins,
        mask_center=mask_center,
        feature_dim=int(len(mu_rt13)),
        tau_margin=float(tau),
        kappa_minsim=float(kappa),
        temperature=float(temperature),
        gamma_fft=float(gamma_fft),
        rt13_protos=[p for p in rt13_protos],
        htr_protos=[p for p in htr_protos],
        rt13_files=rt13_paths,
        htr_files=htr_paths,
    )
    bank = PrototypeBank(mu_rt13=mu_rt13, mu_htr=mu_htr, E_rt13=E_rt13, E_htr=E_htr, meta=meta)

    if out_npz:
        bank.save(out_npz)
        print(f"[OK] bank saved -> {out_npz}")
    print(f"τ={meta.tau_margin:.3f}, κ={meta.kappa_minsim:.3f}, T={meta.temperature:.2f}, γ_fft={meta.gamma_fft:.2f}, k_per_class={k_per_class}")
    print(f"rt13 exemplars used: {len(rt13_paths)} / {len(rt13_paths_all)}  |  htr exemplars used: {len(htr_paths)} / {len(htr_paths_all)}")
    return bank

# ---------------------------
# Prediction helpers (Block A override)
# ---------------------------
def softmax_scores(scores: np.ndarray, T: float = 1.0) -> np.ndarray:
    z = (scores / max(1e-12, T))
    z = z - z.max()
    e = np.exp(z)
    return e / (e.sum() + 1e-12)

def mixture_weights_nnls(f: np.ndarray, protos: List[np.ndarray]) -> np.ndarray:
    P = np.stack(protos, axis=1)  # (D, C)
    w, _ = nnls(P, f)             # (C,)
    s = w.sum()
    return (w / s) if s > 1e-12 else np.ones_like(w)/len(w)

def _class_score_maxproto(f: np.ndarray, protos: List[np.ndarray]) -> float:
    """Score = max cosine similarity to any prototype in the class."""
    if not protos:
        return -1.0
    Ps = np.stack(protos, axis=0)  # (k, D)
    sims = (Ps @ f[0])  # because all are L2-normed, cosine = dot
    return float(sims.max())

def predict_one(image_path: str, bank: PrototypeBank) -> Dict:
    f = extract_features(image_path,
                         mask_center=bank.meta.mask_center,
                         fft_bins=bank.meta.fft_bins,
                         gamma_fft=bank.meta.gamma_fft)
    # Block B: score vs multi-prototypes
    s_rt13 = _class_score_maxproto(f, bank.meta.rt13_protos)
    s_htr  = _class_score_maxproto(f,  bank.meta.htr_protos)
    s_max  = max(s_rt13, s_htr)
    margin = abs(s_rt13 - s_htr)

    # display probabilities (softmax over similarities)
    probs = softmax_scores(np.array([s_rt13, s_htr]), T=bank.meta.temperature)
    p_rt13, p_htr = float(probs[0]), float(probs[1])

    # NNLS mixture weights w.r.t. MEAN prototypes (stable 2D reconstruction)
    w = mixture_weights_nnls(f[0], [bank.mu_rt13, bank.mu_htr])  # [w_rt13, w_htr]

    # Block A: decision rule with NNLS override
    # thresholds gate
    if s_max < bank.meta.kappa_minsim:
        label = "uncertain"
    else:
        if margin >= bank.meta.tau_margin:
            label = "rt13" if s_rt13 > s_htr else "htr"
        else:
            # NNLS override if decisive
            if (w[0] >= 0.60) and (w[0] - w[1] >= 0.20):
                label = "rt13"
            elif (w[1] >= 0.60) and (w[1] - w[0] >= 0.20):
                label = "htr"
            else:
                label = "uncertain"

    return {
        "file": os.path.basename(image_path),
        "label": label,
        "s_rt13": s_rt13,
        "s_htr":  s_htr,
        "s_max":  s_max,
        "margin": margin,
        "p_rt13": p_rt13,
        "p_htr":  p_htr,
        "w_rt13": float(w[0]),
        "w_htr":  float(w[1]),
    }

def predict_folder_with_bank(input_dir: str, bank: PrototypeBank, out_csv: Optional[str] = None) -> List[Dict]:
    paths = list_images(input_dir)
    if not paths:
        raise FileNotFoundError(f"No images found in '{input_dir}'")
    rows = [predict_one(p, bank) for p in paths]
    if out_csv:
        os.makedirs(os.path.dirname(out_csv), exist_ok=True) if os.path.dirname(out_csv) else None
        with open(out_csv, "w", newline="") as f:
            w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
            w.writeheader()
            for r in rows:
                w.writerow(r)
        print(f"[OK] wrote {len(rows)} predictions -> {out_csv}")
    return rows



bank = build_bank_from_dirs(
    htr_dir="/Users/justinmeng/Desktop/Project Quantum/data/STO_ideal_HTR",
    rt13_dir="/Users/justinmeng/Desktop/Project Quantum/data/STO_ideal_RT13",
    out_npz="artifacts/banks/sto_bank_2025_10_21.npz",
    fft_bins=128,
    mask_center=True,
    gamma_fft=1.5,            # Block C
    k_per_class=2,            # Block B multi-prototype
    top_k_rt13=None,          # or an integer like 8 to keep only top-quality
    top_k_htr=None,           # same idea
    val_rt13_dir="/Users/justinmeng/Desktop/Project Quantum/data/Val",
    val_htr_dir="/Users/justinmeng/Desktop/Project Quantum/data/Val",
    temperature=0.30          # Block A: sharper display probs
)

rows = predict_folder_with_bank(
    input_dir="/Users/justinmeng/Desktop/Project Quantum/data/Test",
    bank=bank,
    out_csv="artifacts/results/test_preds.csv"
)
rows


[OK] bank saved -> artifacts/banks/sto_bank_2025_10_21.npz
τ=0.020, κ=0.150, T=0.30, γ_fft=1.50, k_per_class=2
rt13 exemplars used: 21 / 21  |  htr exemplars used: 19 / 19
[OK] wrote 7 predictions -> artifacts/results/test_preds.csv


[{'file': 'HTR_12.png',
  'label': 'htr',
  's_rt13': 0.9802926300323045,
  's_htr': 0.9893016391323114,
  's_max': 0.9893016391323114,
  'margin': 0.009009009100006904,
  'p_rt13': 0.49249305655643444,
  'p_htr': 0.507506943443058,
  'w_rt13': 0.0,
  'w_htr': 1.0},
 {'file': 'HTR_13.png',
  'label': 'htr',
  's_rt13': 0.987136418166538,
  's_htr': 0.9898926331153116,
  's_max': 0.9898926331153116,
  'margin': 0.00275621494877365,
  'p_rt13': 0.49770317036498396,
  'p_htr': 0.5022968296345136,
  'w_rt13': 0.0,
  'w_htr': 1.0},
 {'file': 'HTR_15.png',
  'label': 'htr',
  's_rt13': 0.9766612586196348,
  's_htr': 0.9905737135033581,
  's_max': 0.9905737135033581,
  'margin': 0.01391245488372328,
  'p_rt13': 0.48840836496214934,
  'p_htr': 0.5115916350373391,
  'w_rt13': 0.1740591242334013,
  'w_htr': 0.8259408757665988},
 {'file': 'HTR_16.png',
  'label': 'htr',
  's_rt13': 0.9716627310517841,
  's_htr': 0.9803179406812714,
  's_max': 0.9803179406812714,
  'margin': 0.008655209629487293,
